In [51]:
import pandas as pd
import numpy as np
# connecting with the JavaScript file
from Naked.toolshed.shell import execute_js, muterun_js
import sys

In [2]:
import re, collections

In [3]:
from os.path import dirname, join

In [4]:
def clean_up (file_name ): #, from_file=False):
    rep = {'%20': ' ','%2F%2F':'://', '%2F':'/',';':'\n','.png':'.png \n','%3A':''}
    
   # if from_file:
#         contents = file_name
#         for i, j in rep.items():
#             contents = contents.replace(i, j)
    #else:
    with open(file_name) as infile:
        contents = infile.read()
        # replacements
        for i, j in rep.items():
            contents = contents.replace(i, j)
                
    return contents
        

In [7]:
data=clean_up("./bokeh_app/scripts/Test_results/alexa_test_result.txt")

In [94]:
import os
from os.path import dirname

os.path.dirname(os.path.abspath('__filename__'))

'C:\\Users\\ktadesse\\Desktop\\Project\\Webpageranking'

In [95]:
def prepare(file_name, from_file=False):
    collection = []
    data = collections.defaultdict(dict)
    count_num_data = 0
    
    # checking for expernal or internal file source
    if from_file:
        with open(file_name, 'r') as l_file:
            content = l_file.read()
            print(type(content))
    else:
        content = file_name
#     print(content)
    
#     separate each param and add to list
    line = content.split('\n')
   
    for each in line:
        
        values = each.split(':')
        
        # forming params and value pairs
        if len(values) == 2:
            key, value = values
        elif len(values) == 3:
            key, value_1, value_2 = values
            value = value_1+":"+value_2
            if key == 'web address':
                catagory = value
        else:
            pass
        
        # put the value for each param of single website together
        data[catagory].setdefault(key,[]).append(value)
        #if key not in ['web address', 'Waterfall view']:
           # print data[catagory].setdefault(key)
      
        cleaned_dict = dict(data)
      

    for web in cleaned_dict.keys():
        count_num_data = count_num_data + 1
        #print count_num_data
        #print web
        del cleaned_dict[web]['Waterfall view']
        del cleaned_dict[web]['web address']

        for x in cleaned_dict[web].keys():
            if '' in cleaned_dict[web][x]:
                for y in cleaned_dict[web][x]:
                    cleaned_dict[web][x] = np.nan
        if '' in cleaned_dict[web]:
             del cleaned_dict[web]['']

        else:
            pass
    #print "The number of data collected is: ", count_nam_data

    return cleaned_dict

In [96]:
data =prepare(data)

In [97]:
def select(dataframe, param):
    dfT = dataframe.transpose()
    
    # covert back to frame
    df = pd.DataFrame(data=dfT[param]).dropna()
    
    # this link can be changed to .mean() for avarge
    # .min() for the minmun value
    # .max() for the max value
    s = df.apply(lambda x: pd.Series(x[param]), axis = 1).transpose().max()
    s.name = param
    
    # conver serias to DataFrame
    df_return = df.drop(param, axis=1).join(s)
    
    return df_return
    

In [98]:
df = pd.DataFrame(data)

In [99]:
select(df,'Load time')

,Load time


In [100]:

def table(df):
    params = ['(Doc complete) Byets in', '(Doc complete) Requests','(Fully loaded) Bytes in','(Fully loaded) Requests',
          '(Fully loaded) Time','DOM elements','First byte','Load time','Speed Index','Start render',]
 
    df_list = [select(df, param) for param in params]
    
    for indx, i in enumerate(df_list):
        
        # first time run
        if indx == 0:
            joined = i
        
        if indx < len(df_list) -1:
        
            joined = joined.join(df_list[indx+1])
            
    
       
    joined.index.names = ['Site name']

    return joined
#     df1 = pd.DataFrame(data=df_list[:1])
#     print df1
#     df2 = pd.DataFrame(data=df_list[1:2])
    #return pd.merge(*df_list, how='outer')
    

   

In [101]:
dfnew = table(df)

time = dfnew.drop(['(Doc complete) Byets in','(Fully loaded) Bytes in','(Fully loaded) Requests','(Doc complete) Requests'], axis = 1)
request = dfnew[['(Fully loaded) Requests','(Doc complete) Requests']]
byets = dfnew[['(Doc complete) Byets in','(Fully loaded) Bytes in']]
#time

In [102]:
dfnew

,(Doc complete) Byets in,(Doc complete) Requests,(Fully loaded) Bytes in,(Fully loaded) Requests,(Fully loaded) Time,DOM elements,First byte,Load time,Speed Index,Start render
Site name,,,,,,,,,,


In [52]:
data = pd.read_csv("D://Project//Webpageranking/bokeh_app/data/alexasite.csv")

In [54]:
df =pd.DataFrame(data=data)

In [103]:
# dfnew.to_csv('./bokeh_app/data/formatetop200.csv')

## we can start making the model now

## Algorithm:

### Step1: Find min and max of each of the 7 parameters.

In [87]:
df_set = pd.read_csv('./bokeh_app/data/formatetop100.csv')

params = ['(Doc complete) Byets in', '(Doc complete) Requests','(Fully loaded) Bytes in','(Fully loaded) Requests',
          '(Fully loaded) Time','DOM elements','First byte','Load time','Speed Index','Start render',]
# list 
param_values = {}
for param in params:
    param_values.update({
    param:[df_set[param].min(), df_set[param].mean(), df_set[param].max()],})


# for key in param_values:
#     print param_values[key]

df_param = pd.DataFrame.from_dict(param_values, orient='index')
df_param.columns=['Min','Average','Max']
#df_param
#df_param = pd.DataFrame(list(param_values.items()),columns=['Min','Average','Max'])


FileNotFoundError: File b'./bokeh_app/data/formatetop100.csv' does not exist

In [88]:
df_set.describe().transpose()

NameError: name 'df_set' is not defined

### Step2: Normalize each parameter. After normalization, each parameter will have a value in between 0 and 1. 

If you want to normalize you data you can do as you suggest and simply calculate:
                 $$ zi=xi−min(x)/max(x)−min(x) $$
where x=(x1,...,xn) and zi is now your ith normalized data. As a proof of concept (although you did not ask for it) here is some R code and accompanying graph to illustrate this point:

In [89]:
def normalize(x, param, df_param):
    min_val = df_param.loc[param][0]
    max_val = df_param.loc[param][2]
    
    return (x - min_val)/(max_val - min_val)
    

In [90]:
normalize(700, 'Speed Index', df_param)

NameError: name 'df_param' is not defined

######  Step 3: iterate through each parameter and assign it points (from 1 to 100) on the basis of its normalized scale. For example, after normalization speed index is 0.2 for one website. Thinking intuitively, speed value should be more for good ranking and good performance. Now, we design our scale from 0 to 1 in such a way that high speed = high points. Example 1: calculating speed index

In [91]:
from sklearn import preprocessing

def normal(df_set, name):
    
    x = df_set

    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    #df = pd.DataFrame(x_scaled, columns= ['(Doc complete) Byets in', '(Doc complete) Requests','(Fully loaded) Bytes in','(Fully loaded) Requests',
              #'(Fully loaded) Time','DOM elements',b'First byte','Load time','Speed Index','Start render',])
    #df=df.set_index(df_set['Site name'])
    df = pd.DataFrame(x_scaled, columns=[df_set.columns])
   
    df.to_csv('./bokeh_app/data/'+name+'.csv')
    return df

ModuleNotFoundError: No module named 'sklearn'

In [56]:
nor=normal(df,"nor_time")

In [59]:
web =nor.reset_index()

In [124]:
test = web['Speed Index'].drop(labels=[8]).transpose().iloc[0]

In [70]:
df1 = test.convert_objects(convert_numeric=True)

C:\Users\ktadesse\Miniconda3\envs\Webranking\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


In [125]:
test

0     0.151825
1     0.810258
2     0.279421
3     0.115560
4     0.110289
5     0.344006
6     0.146948
7     0.319226
9     0.319226
10    0.371067
11    0.088499
12    0.094635
13    0.655129
14    0.112177
15    0.178965
16    0.057347
17    0.028084
18    0.029185
19    0.031702
20    0.181246
21    0.107458
22    0.073474
23    0.201542
24    0.107064
25    0.354783
26    0.129405
27    0.505664
28    0.284928
29    0.484031
30    0.815686
        ...   
67    0.166142
68    0.021712
69    0.107930
70    0.228839
71    0.031466
72    0.023678
73    0.024072
74    0.127596
75    0.007867
76    0.191551
77    0.187146
78    0.607300
79    0.000000
80    0.113436
81    0.123112
82    0.028713
83    0.132159
84    0.120673
85    0.458858
86    0.890733
87    0.125865
88    0.380979
89    0.676998
90    0.125000
91    0.389711
92    0.320957
93    0.136957
94    0.415120
95    0.057898
96    0.199025
Name: (Speed Index,), Length: 96, dtype: float64

In [123]:
# test.columns.droplevel(['Speed Index', 'Start render'])

In [122]:
test.drop(labels=[8]).transpose().iloc[0]

0     0.151825
1     0.810258
2     0.279421
3     0.115560
4     0.110289
5     0.344006
6     0.146948
7     0.319226
9     0.319226
10    0.371067
11    0.088499
12    0.094635
13    0.655129
14    0.112177
15    0.178965
16    0.057347
17    0.028084
18    0.029185
19    0.031702
20    0.181246
21    0.107458
22    0.073474
23    0.201542
24    0.107064
25    0.354783
26    0.129405
27    0.505664
28    0.284928
29    0.484031
30    0.815686
        ...   
67    0.166142
68    0.021712
69    0.107930
70    0.228839
71    0.031466
72    0.023678
73    0.024072
74    0.127596
75    0.007867
76    0.191551
77    0.187146
78    0.607300
79    0.000000
80    0.113436
81    0.123112
82    0.028713
83    0.132159
84    0.120673
85    0.458858
86    0.890733
87    0.125865
88    0.380979
89    0.676998
90    0.125000
91    0.389711
92    0.320957
93    0.136957
94    0.415120
95    0.057898
96    0.199025
Name: (Speed Index,), Length: 96, dtype: float64

--------------------------------------------------------------------------------------------------------------------------------
$$  If (speed > =0 and speed <= .1) {  $$  
$$  //assign lowest points to the speed parameter variable. Eg = assign 10 points $$ 
$$  If(speed >= 0.2 and speed =< .3){ $$ 
$$  //Assign 20 points $$ 
$$  If(speed > = .3 and speed =<.4 $$ 
$$  //Assign 30 points $$ 
$$  And so on – for speed index - the points we assign increases as the value of the parameter lies in $$ 
$$  the range closer to 1. $$ 

Example 2: calculating load time
If (load time > =0 and speed <= .1) {

//assign highest points to the speed parameter variable. Eg = assign 90 points
If(speed >= 0.1 and speed =< .2){
// Assign 80 points
If(speed > = .2 and speed =<.3)
//Assign 70 points
And so on .. for load time - the points decreases as the value of the parameter lying in the range
is closer to 0.
------------------------------------------------------------------------------------------------------------------------------------------

Likewise, we do this step for each parameter for each website.
At the end of this step, we get total optimal points for each website having: speed points + load
time points + ...all other parameter points.

Step 4:
Now, we have the total points depending on the scale of each of the 100 websites calculated.

Step 5:
Compare the optimal points calculated now to get the ranking. As we have normalized the scale,
and have the model having total optimal points for each website.

Step 6:
Sort the total points of each website and get the ranking info depending on the sort. One with
the maximum points calculated is the ranked first and so on..

Step 7 : Once we have the ranking, if user wants to improve the ranking of a given website, so
basically increase the points for that website.

//Calculate increase of points required to make a website No.1
Points_rank1_website = p1;
Current_ Points_of_website_user_wants_to_improve = p2;
Points_needed = (points of rank 1 website – point of website user wants to improve to rank 1)
//Then increase those many points

Points_of_website_user_wants_to_improve = Points_of_website_user_wants_to_improve +
points_needed;

Step 7:
Depending on the new points, again sort the list and display ranking. Now user has the provision
to improve the ranking of any website and display it.